In [1]:
# pip install 'git+https://github.com/facebookresearch/detectron2.git' intstall in console alongside visual studio build tool c++ to run detectron2 wihtin env

ERROR: Invalid requirement: "'git+https://github.com/facebookresearch/detectron2.git'": Expected package name at the start of dependency specifier
    'git+https://github.com/facebookresearch/detectron2.git'
    ^


In [2]:

import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

%matplotlib inline

# import some common libraries
import torch
import numpy as np
import cv2
from pathlib import Path
import shutil
from PIL import Image
from tqdm import tqdm
from matplotlib import image as mpimg
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
import time

In [3]:
%matplotlib inline

In [4]:
def segment_detectron2(data, ndata, cfg, mode):
    if ndata == 0:
        ndata = data.shape[0]
    D = []
    predictor = DefaultPredictor(cfg)
    SE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    t = time.time()
    for i in range(ndata):
        new_im = data[i]
        outputs = predictor(new_im)
        category = outputs["instances"].pred_classes
        mask = outputs["instances"].pred_masks.to("cpu").numpy().astype(bool)
        Z = np.zeros((mask.shape[1], mask.shape[2]), dtype="uint8")
        for idx in range(mask.shape[0]):
            if (mask[idx].sum() < 2000) or (mode == 'phase'):
                msk = (np.uint8(category[idx].cpu()) + 1) * mask[idx]
                if mode == 'phase':
                    msk = cv2.erode(msk.astype('float32'), SE, iterations=1)
                Z = Z * (~(msk > 0)) + msk
        D.append(Z)
        elapsed = time.time() - t
        print('Time required for processing all data: ', elapsed)
        clear_output(wait=True)
        print('frame ', i + 1, '/', ndata)
    D = np.array(D)
    print('Completed')
    return D

In [1]:
# input parameters
imsize = 768  # 768, 1024
image_format = 'png'  # e.g .png, .svg, etc.
img_dpi = 1200

source3 = r"PHASE"

#num_images = 63
sorted_list_dataset = os.listdir(source3)
sorted_list_dataset.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

print(sorted_list_dataset)

Dataset = []
for idx in tqdm(sorted_list_dataset):
    filepath = os.path.join(source3, idx)
    if filepath.endswith(('.', image_format)):
        file = mpimg.imread(filepath)
        file = cv2.resize(file, (imsize, imsize))
        Dataset.append(file)
        
print(Dataset)
Dataset = np.array(Dataset)
Dataset = ((Dataset - Dataset.min()) / Dataset.max()) * 255
Dataset = np.uint8(Dataset)

print('training set shape', Dataset.shape)

print('training set max', Dataset.max())

img = Dataset[0]
plt.imshow(img)
plt.show()

NameError: name 'os' is not defined